In [22]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np 
import random
import os
import io # ?
import re



In [23]:
filename = "Dost_notepad.txt"
text = open(filename, 'r', encoding='utf-8').read()
text = text.lower()
text = re.sub('[!#()*:;?018\-\'\"\n]', '', text)
chars = sorted(list(set(text)))
print(chars)
print("Символьная длина текста:", len(text))
print("Всего символов:", len(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i, c in enumerate(chars))


length = 50 # Длина последовательности символов
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - length, step): 
    sentences.append(text[i : i + length])
    next_chars.append(text[i+length])
    
print('Число последовательностей символов:', len(sentences))

#One-Hot Encoding
x = np.zeros((len(sentences), length, len(chars)), dtype = np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, fragment in enumerate(sentences):
    for j, char in enumerate(fragment):
        x[i, j, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

[' ', ',', '.', 'a', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
Символьная длина текста: 227714
Всего символов: 54
Число последовательностей символов: 227664


In [30]:
#models 

def lstm_model():
    model = keras.Sequential()
    model.add(keras.Input(shape = (length, len(chars))))
    #model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(128, return_sequences=False, stateful=False))
    model.add(layers.Dense(len(chars), activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
    
              

In [31]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
model_1 = lstm_model()

weights_path = 'C:\\Users\\Pankratz\\Desktop\\Sequence_generation\\weights'
epochs = 20
batch_size = 128
#checkpoint = ModelCheckpoint(weights_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

for epoch in range(epochs):
    model_1.fit(x, y, epochs=1, batch_size=batch_size)
    start = random.randint(0, len(text) - length - 1)
    model_output = ""
    fragment = text[start : start + length]
    print('Генерируем начиная с отрывка: "' + fragment + '"')

    diversity = 0.7
    for i in range(90):
        x_pred = np.zeros((1, length, len(chars)))
        for j, char in enumerate(fragment):
            x_pred[0, j, char_to_int[char]] = 1.0
            prediction = model_1.predict(x_pred, verbose = 0)[0]
            index = sample(prediction, diversity)
            next_char = int_to_char[index]
            fragment = fragment[1:] + next_char
            model_output += next_char
        
    print('Сгенерированный текст:', model_output)
    print()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
227664/227664 [==============================] - 227s 997us/sample - loss: 2.6251
Генерируем начиная с отрывка: "иваюсь, что я в чемто у  вас  прощенья прошу.. я у"
Сгенерированный текст: ооо   и мае н     о  а  удло    оом      ,е тд  о и,  вая о е  тс    ласа очодт ееео  е  и  еит оеме лв нусоа   е ее   бое о,е иивтви  ек ну    сяоир онорт    н  о ытто з е мдло л   о тсд н к      нкик  нвоя яад юв  н л т р овоис и и и     в евя     к в е зсо аоокаан    н е,тм коо у    е ии    т ч    и оо      изы   мбяои ат    кво     оо ичо и  онил      а  мо  ееиикн   ко . ир аи  имнт у я ти иити  и еяаоян  ли ат   йе  ес  уа     е тно олаав  о щ  тиуэ ед ж   тсеосимомлю  илр ч  иототсь ц  тву    т гоян  к,. еи еоян интоа    я  о и и ее    а оиа  онос    е, у  ео таияа  ооо ар ша  уин  т и о л     а ад твыв еиеу ея   д ач нуо  ч    ве   е  и , хуо    а  еад в  ои   о ансс д яп д ти  о     лд еоол ,нв ато у

227664/227664 [==============================] - 224s 983us/sample - loss: 2.3788
Генерируем начиная с отрывка: "чезала. кто знает, может быть, ее у меня никогда и"
Сгенерированный текст:  к  нспск    оад д  суе иево рал з мнакт  яуаом д тж   с.о в  с   о   ое р к  едотвтунтн и ,  и тоов о, о   иу   равт.хя дурпмн  о с а  е ярра  фмое о   и и  т  оеслвта  .и  п чо кляи боклрпо  ялна, б  ом  нооредем с о в    опр   ек  вевто  хиа  по ыут  им   шужмеск зн   еита  и ичвн ,ка    к неяобк  с иг си  ч  д апегн  с    охс  оип.    о чота оо  ооок  т мтл с о ткя    пип он т     ати фян   оч  ео   нно лч  е ч еовонну к  нип  ньитв д   з   е        ект е е о ми , осюм к писи мл  . уке  меенев о   ны нт ы.иа сс ,  тя л ,о  снп речня но  н  о к  нжид  к ччпоыр к  си с  д  уод о   с  еирв   чни од   мннн о е   н   кв оекм  мд,ч с ,  слотр с .в но , иг о оа е  к   сри  и   осео   од  и екае .симесв  трс   в  р  кын  ч д дссооо еияп  ам ои д  мммрвпо   . унвт   и  ва р ,аср   ге  и    аеи е от   ст вл

227664/227664 [==============================] - 231s 1ms/sample - loss: 2.2896
Генерируем начиная с отрывка: "оспода завтра же, сообща, отъезжавшему далеко  в  "
Сгенерированный текст: е  аенианмтсвпдев у а ыватно в  ееесир е.к,п а ов ем оот.влметнуоитеж с еа еу с ренм рдрв   о  в ае о нк ее  чг  нн   ио,ег беояаноен  оп лоб е   е онм ена е  е чо ааел о н о  лео снтог  ир а ен лвуое г аеемеоо а е уоомред а чон   н в  мнес а  т.у,и е чра .иону  ти  оео .яекеосекиичытэеовнмнн    нн  е а е  и  ното  е,ооаклубт  уоитив  к вч   н с ш е ке е  а ое ооитжс  еи ьно ,ину    аоеи ет .аноегн.снонд   оабоеег  а ия псе,и  .м  е роз  в  хвяаоо пеосб   л ь  сооадгшброогкоеик с ноенри    н, .  вр ло осрбмв н н  аеыв о  лошеярлоквг по у  умоп ае т, т е е олер бмапкилн ллнтне   н  дии  омни нл аи  с    е   ге,о  ежосо нсеао  йв.е с   ен ки   у   кооо н.ооу. р о  ее о пнеу   квпрен  ио нвннетаиегоуесун   е о да  авкксебк ур гьрн н лвсас  ж  ,ка гкякооев  нооега нило жккиу   т уаовс ыиошонос уст  еов    д

133504/227664 [================>.............] - ETA: 1:36 - loss: 2.2365